In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import seaborn as sns

from skspatial.objects import Line, Plane
from skspatial.plotting import plot_3d


from skspatial.objects import Line, Cylinder, Point
from skspatial.plotting import plot_3d

import phasespace

import tensorflow


In [ ]:
print(phasespace.__version__)

print(tensorflow.__version__)

# Utility functions

In [ ]:
def mag(p3):
  #print(p3)
  p = np.sqrt(p3[0]*p3[0] + p3[1]*p3[1] + p3[2]*p3[2])
  #if p<5000:
  #  print(p3,p)
  return p

def invmass(p4s):
  E,px,py,pz = 0,0,0,0

  for p4 in p4s:
    E += p4[3]
    px += p4[0]
    py += p4[1]
    pz += p4[2]

  m2 = E**2 - (px**2 + py**2 + pz**2)
  if m2>=0:
    return np.sqrt(m2)
  else:
    return -np.sqrt(-m2)

def invmass_cols(p4):

  E = p4[3]
  px = p4[0]
  py = p4[1]
  pz = p4[2]

  m2 = E**2 - (px**2 + py**2 + pz**2)
  m = -999*np.ones(len(E))
  mask = m2>=0
  #print(mask[mask])
  #print(mask[~mask])

  m[mask] = np.sqrt(m2[mask])
  m[~mask] = -np.sqrt(-m2[~mask])

  return m


def opening_angle(p4s):
  p0mag = np.sqrt(p4s[0][0]**2 + p4s[0][1]**2 + p4s[0][2]**2)
  p1mag = np.sqrt(p4s[1][0]**2 + p4s[1][1]**2 + p4s[1][2]**2)

  dot_product = p4s[0][0]*p4s[1][0] + p4s[0][1]*p4s[1][1] + p4s[0][2]*p4s[1][2]

  theta = np.arccos(dot_product/(p0mag*p1mag))

  return theta


def distance(v1, v2):
  dx = v1[0] - v2[0]
  dy = v1[1] - v2[1]
  dz = v1[2] - v2[2]

  d = np.sqrt(dx**2 + dy**2 + dz**2)
  return d

##################################################################


# Generate decays for center-of-Earth model

In [ ]:
# Loop over different values

decays = {}
decays['M_DM'] = []
decays['M_A'] = []

decays['px_mu1'] = []
decays['py_mu1'] = []
decays['pz_mu1'] = []
decays['e_mu1'] = []

decays['px_mu2'] = []
decays['py_mu2'] = []
decays['pz_mu2'] = []
decays['e_mu2'] = []

#MASS_A = [250,1000,5000]
#MUON_MASS = 105.11

MASS_A = [.250,1,5]
DM_MASSES = [10,100,1000]

# High mass states
#MASS_A = [.220,0.5, 0.75, 1]
#DM_MASSES = [1000,10000,100000, 1000000]

MUON_MASS = 0.10511

thetas = []
pmags = []

muon_p = []

nevents_to_generate = 10000

#pmags_GeV = [10,100,1000]

for MASS_A in MASS_A:
  for pmag in DM_MASSES:

    #pmag = pmag_GeV*1000

    print(MASS_A,pmag)

    boost_vector = np.array([0,0, pmag, np.sqrt(pmag**2 + MASS_A**2)])
    boost_vectors = np.tile(boost_vector, (nevents_to_generate,1))


    weights, particles = phasespace.nbody_decay(MASS_A,
                                                [MUON_MASS, MUON_MASS]).generate(n_events=nevents_to_generate, boost_to=boost_vectors)

      
    p0 = particles['p_0'][:].numpy().T
    p1 = particles['p_1'][:].numpy().T

    #data[MASS_A][pmag] = [p0.T, p1.T]
    decays['M_DM'] += (pmag*np.ones(nevents_to_generate)).tolist()
    decays['M_A'] += (MASS_A*np.ones(nevents_to_generate)).tolist()

    #print(len(decays['M_A']))

    decays['px_mu1'] += p0[0].tolist()
    decays['py_mu1'] += p0[1].tolist()
    decays['pz_mu1'] += p0[2].tolist()
    decays['e_mu1'] +=  p0[3].tolist()

    decays['px_mu2'] += p1[0].tolist()
    decays['py_mu2'] += p1[1].tolist()
    decays['pz_mu2'] += p1[2].tolist()
    decays['e_mu2'] +=  p1[3].tolist()

dfdec1 = pd.DataFrame.from_dict(decays)

# Generate a few other entries

# pmag, theta (degrees), phi
px1 = dfdec1['px_mu1'].values
py1 = dfdec1['py_mu1'].values
pz1 = dfdec1['pz_mu1'].values
e1 = dfdec1['e_mu1'].values

pmag1 = mag([px1,py1,pz1])
theta1 = np.rad2deg(np.arccos(pz1/pmag1))
phi1 = np.arctan2(py1,pz1)

dfdec1['pmag1'] = pmag1
dfdec1['theta1'] = theta1
dfdec1['phi1'] = phi1

px2 = dfdec1['px_mu2'].values
py2 = dfdec1['py_mu2'].values
pz2 = dfdec1['pz_mu2'].values
e2 = dfdec1['e_mu2'].values

pmag2 = mag([px2,py2,pz2])
theta2 = np.rad2deg(np.arccos(pz2/pmag2))
phi2 = np.arctan2(py2,pz2)

dfdec1['pmag2'] = pmag2
dfdec1['theta2'] = theta2
dfdec1['phi2'] = phi2

# Opening angle
p4s = [[px1,py1,pz1,e1], [px2,py2,pz2,e2]]
thetas = np.rad2deg(opening_angle(p4s))
dfdec1['opening angle'] = thetas

dfdec1.sample(5)

In [ ]:
dfdec1['opening angle']


In [ ]:
plt.hist(dfdec1['opening angle'], bins=100);


In [ ]:
palette = sns.color_palette("bright",3)

g = sns.displot(dfdec1, x='opening angle', hue='M_DM', col='M_A', bins=50, palette=palette)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{DM}$ (GeV/c$^2$)")
for text in g.legend.texts:
    text.set_fontsize(20)

In [ ]:
g = sns.displot(dfdec1, x='opening angle', col='M_DM', hue='M_A', bins=50, palette=palette)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)



axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

In [ ]:
g = sns.displot(dfdec1, x='theta2', col='M_DM', hue='M_A', bins=50, palette=palette, fill=True)#, height=5, aspect=1.5, fill=True)

g.set(ylim=(0,50000))

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

In [ ]:
g = sns.displot(dfdec1, x='pmag1', col='M_DM', hue='M_A', bins=50, palette=palette, facet_kws=dict(sharey=False, sharex=False))

for ax in g.axes.flat:
  ax.ticklabel_format(axis="x", style="plain", scilimits=(0,0))

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (GeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

In [ ]:
# Loop over different values

decays = {}
decays['M_DM'] = []
decays['M_A'] = []

decays['px_mu1'] = []
decays['py_mu1'] = []
decays['pz_mu1'] = []
decays['e_mu1'] = []

decays['px_mu2'] = []
decays['py_mu2'] = []
decays['pz_mu2'] = []
decays['e_mu2'] = []

MASS_A_GEV = [.250,1,5]
DM_MASSES = [10,100,1000]

MUON_MASS = 0.10511

thetas = []
pmags = []

muon_p = []

nevents_to_generate = 1000

for MASS_A in MASS_A_GEV:
  for DM_MASS in DM_MASSES:

    print(MASS_A,DM_MASS)

    muon_plus1 = phasespace.GenParticle('mu+1', MUON_MASS)
    muon_minus1 = phasespace.GenParticle('mu-1', MUON_MASS)

    muon_plus2 = phasespace.GenParticle('mu+2', MUON_MASS)
    muon_minus2 = phasespace.GenParticle('mu-2', MUON_MASS)

    dgamma1 = phasespace.GenParticle('Dgamma1', MASS_A).set_children(muon_plus1, muon_minus1)
    dgamma2 = phasespace.GenParticle('Dgamma2', MASS_A).set_children(muon_plus2, muon_minus2)

    dm_pair = phasespace.GenParticle('DM_pair', 2*DM_MASS).set_children(dgamma1, dgamma2)

    weights, particles = dm_pair.generate(n_events=nevents_to_generate)

    # Grab both pairs
    p0 = particles['mu+1'][:].numpy().T
    p1 = particles['mu-1'][:].numpy().T
    p2 = particles['mu+2'][:].numpy().T
    p3 = particles['mu-2'][:].numpy().T

    # We have twice as many entries because we're going to use both pairs
    # of muons
    decays['M_DM'] += (DM_MASS*np.ones(2*nevents_to_generate)).tolist()
    decays['M_A'] += (MASS_A*np.ones(2*nevents_to_generate)).tolist()

    #print(len(decays['M_A']))

    decays['px_mu1'] += p0[0].tolist() + p2[0].tolist()
    decays['py_mu1'] += p0[1].tolist() + p2[1].tolist()
    decays['pz_mu1'] += p0[2].tolist() + p2[2].tolist()
    decays['e_mu1'] +=  p0[3].tolist() + p2[3].tolist()

    decays['px_mu2'] += p1[0].tolist() + p3[0].tolist()
    decays['py_mu2'] += p1[1].tolist() + p3[1].tolist()
    decays['pz_mu2'] += p1[2].tolist() + p3[2].tolist()
    decays['e_mu2'] +=  p1[3].tolist() + p3[3].tolist()

dfdec2 = pd.DataFrame.from_dict(decays)

# Generate a few other entries

# pmag, theta (degrees), phi
px1 = dfdec2['px_mu1'].values
py1 = dfdec2['py_mu1'].values
pz1 = dfdec2['pz_mu1'].values
e1 = dfdec2['e_mu1'].values

pmag1 = mag([px1,py1,pz1])
theta1 = np.rad2deg(np.arccos(pz1/pmag1))
phi1 = np.arctan2(py1,pz1)

dfdec2['pmag1'] = pmag1
dfdec2['theta1'] = theta1
dfdec2['phi1'] = phi1

px2 = dfdec2['px_mu2'].values
py2 = dfdec2['py_mu2'].values
pz2 = dfdec2['pz_mu2'].values
e2 = dfdec2['e_mu2'].values

pmag2 = mag([px2,py2,pz2])
theta2 = np.rad2deg(np.arccos(pz2/pmag2))
phi2 = np.arctan2(py2,pz2)

dfdec2['pmag2'] = pmag2
dfdec2['theta2'] = theta2
dfdec2['phi2'] = phi2

# Opening angle
p4s = [[px1,py1,pz1,e1], [px2,py2,pz2,e2]]
thetas = np.rad2deg(opening_angle(p4s))
dfdec2['opening angle'] = thetas

dfdec2.sample(5)

In [ ]:
palette = sns.color_palette("bright",3)

g = sns.displot(dfdec2, x='opening angle', hue='M_A', col='M_DM', bins=50, palette=palette)

g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
# Legend texts
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

In [ ]:
g = sns.displot(dfdec2, x='opening angle', col='M_DM', hue='M_A', bins=50, palette=palette)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)


In [ ]:
#fig=plt.figure(figsize=(12,4))
#ax = fig.add_subplot(1,1,1)
#ax.ticklabel_format(style='plain', axis='both')

g = sns.displot(dfdec2, x='pmag1', col='M_DM', hue='M_A', bins=50, palette=palette, facet_kws=dict(sharey=False, sharex=False))

for ax in g.axes.flat:
  ax.ticklabel_format(axis="x", style="plain", scilimits=(0,0))

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (GeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

In [ ]:
g = sns.displot(dfdec2, x='theta2', col='M_DM', hue='M_A', bins=50, palette=palette, fill=True)#, height=5, aspect=1.5, fill=True)

#g.set(ylim=(0,5000))

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

In [ ]:
dfdec2['costheta2'] = np.cos(np.deg2rad(dfdec2['theta2'].values))

In [ ]:
g = sns.displot(dfdec2, x='costheta2', col='M_DM', hue='M_A', bins=50, palette=palette, fill=True)#, height=5, aspect=1.5, fill=True)

#g.set(ylim=(0,5000))

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

# Use these decays to see if they hit CMS

In [ ]:
#'''
import pickle
# Try the pickle file
#eloss_data = pickle.load(open('energies_and_distances_file.pkl','r'))

with open('energies_and_distances_file.pkl', 'rb') as f:
    # Load the pickled object from the file
    eloss_data = pickle.load(f)

eloss_interp = eloss_data['interp']
#'''

In [ ]:
eloss_interp(1000,100)[0][0]

In [ ]:
df = dfdec2.iloc[1]

df

In [ ]:
#dftmp.iloc[1]
print(dfdec1['pmag1'].unique())

In [ ]:
def throw_muons_at_CMS(df_input, ndecays=None, MAKE_PLOTS=False):

  # Define CMS
  origin_CMS = [0, 0, 0]

  nmuons = 0

  # CMS, units are meters. x is direction of beam and z is up
  cylinder = Cylinder.from_points([-10.5, 0, 8], [10.5, 0, 8], 7.5)

  if MAKE_PLOTS:
    fig1 = plt.figure(figsize=(6,6))
    ax1 = fig1.add_subplot(1,1,1,projection='3d')

    fig2 = plt.figure(figsize=(12,12))
    ax2 = fig2.add_subplot(1,1,1,projection='3d')

    fig3 = plt.figure(figsize=(4,4))
    ax3 = fig3.add_subplot(1,1,1)

    # Draw CMS
    cylinder.plot_3d(ax2, alpha=0.2)

  # Which to use?
  #mask = (dfdec2['DM_MASS']==100)
  #mask = mask & (dfdec2['M_A']==5)
  #dftmp = dfdec2[mask]

  #mask = (dfdec1['pmag']==1000000)
  #mask = mask & (dfdec1['M_A']==5000)
  #dftmp = dfdec1[mask]

  dftmp = df_input

  if ndecays is None:
    ndecays = len(dftmp)
  print(f"Will run over {ndecays} decays")

  distances = []
  pmag_origin = []
  pmag_cms = []

  # Range over which to generate the interaction where the muon pairs are created
  limits = 100
  xlo, xhi = -limits,limits
  ylo, yhi = -limits,limits
  zlo, zhi = -limits, -1

  xwidth = xhi-xlo
  ywidth = yhi-ylo
  zwidth = zhi-zlo

  # Generate the points
  #npts = 3000
  #for i in dftmp.index[0:npts]:
  for i in range(0,ndecays):

    #print(i)

    if i%10000==0:
      print(i)

    # Generate the interaction point of the muon pairs
    origin = [xhi-xwidth*np.random.random(), yhi-ywidth*np.random.random(), -zhi-zwidth*np.random.random()]

    # For debugging
    #origin = [0, 0, -10]

    # Print all the origins
    #Point(origin).plot_3d(ax1, c='k')

    df = dftmp.iloc[i]

    pmag = None

    # Loop over the pairs
    for j in [0,1]:
      nmuons += 1
      # Need to mix up z and y
      if j==0:
        px1,py1,pz1 = df['px_mu1'],df['py_mu1'],df['pz_mu1']
        pmag = df['pmag1']
        dir = np.array([px1, py1, pz1])
      else:
        px2,py2,pz2 = df['px_mu2'],df['py_mu2'],df['pz_mu2']
        pmag = df['pmag2']
        dir = np.array([px2, py2, pz2])

      # Skip downward traveling muons
      # Skip downward traveling muons
      if dir[2]<0:
        #print("skipping ",j,dir)
        continue

      # Need to do this to draw the lines correction
      m = mag(dir)
      dir /= m
      dir *= xwidth

      #print("origin")
      #print(origin)
      #print('dir')
      #print(dir)

      line = Line(point=origin, direction=dir) # Point and direction vector

      #print("here")
      point_a,point_b = None, None
      try:
        point_a, point_b = cylinder.intersect_line(line, infinite=False)
      except ValueError:
        1
        #print('does not')

      #print('which:  ',j)
      #print('dir:    ',dir)
      #print('origin: ', origin)
      #print('points: ',point_a, point_b)

      if point_a is not None:
        1
        #point_a.plot_3d(ax2, c='r',s=10)
        #line.plot_3d(ax2, c='k'),#, t_2=100),
        #Point(origin).plot_3d(ax2, c='b')
        #print(origin)
        #print(dir)

      if point_b is not None:
        1
        #point_b.plot_3d(ax2, c='g',s=10)

      if point_b is None or point_a is None:
        1
        #line.plot_3d(ax2, c='y', linestyle='--'),#, t_2=100),
        #Point(origin).plot_3d(ax2, c='y')
      else:
        d = distance(point_a, origin)
        distances.append(d)
        pmag_origin.append(pmag)
        pfinal = eloss_interp(pmag,d)[0][0]
        pmag_cms.append(pfinal)

  if MAKE_PLOTS:
    ax2.set_xlim(-20,20)
    ax2.set_ylim(-20,20)
    ax2.set_zlim(-20,20)
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_zlabel('z')

    plt.sca(ax3)
    plt.hist(distances);

  nhits = len(distances)
  print(f"{nhits}  {ndecays}   {nhits/ndecays:.6f}")

  return pmag_origin, pmag_cms, distances, nmuons


mask = (dfdec2['M_DM']==100)
mask = mask & (dfdec2['M_A']==5)
#dftmp = dfdec2[mask]

#mask = (dfdec1['pmag']==1000000)
#mask = mask & (dfdec1['M_A']==5000)
#dftmp = dfdec1[mask]

pmag_origin, pmag_cms, distances, ndecays = throw_muons_at_CMS(dfdec2[mask][0:10])#,ndecays=10000)

In [ ]:
MASS_A_GEV = [.250,1,5]
DM_MASSES = [10,100,1000]

interactions = {}

#d = dfdec1
#interactions1 = {}

d = dfdec2

interactions = {}
interactions['M_A'] = []
interactions['M_DM'] = []
interactions['pmag_origin'] = []
interactions['pmag_cms'] = []
interactions['distances'] = []
interactions['nmuons'] = []
interactions['model'] = []

for imodel,d in enumerate([dfdec1, dfdec2]):

  for M_A in MASS_A_GEV:

    for M_DM in DM_MASSES:
      print(M_A, M_DM)
      mask = (d['M_DM']==M_DM)
      mask = mask & (d['M_A']==M_A)
      pmag_origin, pmag_cms, distances, nmuons = throw_muons_at_CMS(d[mask][0:10])#,ndecays=1000)

      nentries = len(distances)
      interactions['model'] += (imodel*np.ones(nentries,dtype=int)).tolist()
      interactions['M_A'] += (M_A*np.ones(nentries)).tolist()
      interactions['M_DM'] += (M_DM*np.ones(nentries)).tolist()
      interactions['pmag_origin'] += pmag_origin
      interactions['pmag_cms'] += pmag_cms
      interactions['distances'] += distances
      interactions['nmuons'] += (nmuons*np.ones(nentries,dtype=int)).tolist()

      #interactions2[M_A][M_DM] = {'pmag_origin':pmag_origin, 'pmag_cms':pmag_cms, 'distances':distances, 'nmuons':nmuons}

df_interactions = pd.DataFrame.from_dict(interactions)

df_interactions.to_parquet('df_interactions_v2.gzip', compression='gzip')

df_interactions

In [ ]:
df_int = pd.read_parquet('df_interactions_v2.gzip')

df_int

In [ ]:
filter = df_int['model'] == 0
filter = filter & (df_int['pmag_cms'] > 5)

g = sns.displot(df_int[filter], x='pmag_cms', col='M_DM', hue='M_A', bins=50, palette=palette, \
                common_bins=False, fill=True, facet_kws=dict(sharey=False, sharex=False))#, height=5, aspect=1.5, fill=True)

#g.set(ylim=(0,5000))

# Legend texts
g.legend.get_title().set_fontsize(20)
g.legend.set_title(r"M$_{A'}$ (MeV)")
for text in g.legend.texts:
    text.set_fontsize(20)

axes = g.axes.flat
axes[0].set_title(r'M$_{DM}$ = 10 GeV', fontsize=20)
axes[1].set_title(r'M$_{DM}$ = 100 GeV', fontsize=20)
axes[2].set_title(r'M$_{DM}$ = 1000 GeV', fontsize=20)

In [ ]:
df_int.columns

In [ ]:
MASS_A_GEV = [.250,1,5]
DM_MASSES = [10,100,1000]

dict_eff = {}
dict_eff['M_A'] = []
dict_eff['M_DM'] = []
dict_eff['nmuons'] = []
dict_eff['model'] = []
dict_eff['eff'] = []
dict_eff['# interactions'] = []

df = df_int

for model in [0,1]:

  for M_A in MASS_A_GEV:


    for M_DM in DM_MASSES:

      filter = (df['M_A']==M_A) & (df['M_DM']==M_DM) & (df['model']==model)

      nhits = len(df[filter]['distances'])
      #print(df[filter])
      print(len(df[filter]['nmuons']))
      nmuons = df[filter]['nmuons'].iloc[0]

      eff = nhits/nmuons
      nint = round(1/eff,1)

      dict_eff['M_A'].append(M_A)
      dict_eff['M_DM'].append(M_DM)
      dict_eff['nmuons'].append(nmuons)
      dict_eff['model'].append(model)
      dict_eff['eff'].append(eff)
      dict_eff['# interactions'].append(nint)

      print(f"{model}  {M_A:8.3f}   {M_DM:5d}  {nmuons}   {nhits}     {eff:.6f}")

df_eff = pd.DataFrame.from_dict(dict_eff)


df_eff



In [ ]:
#!pip install dataframe_image
#!pip install selenium

import dataframe_image as dfi


In [ ]:
model = 1

filter = df_eff['model']==model
cols = ['M_A', 'M_DM', '# interactions']

df = df_eff[filter][cols]

dfi.export(df,f"ninteractions_table_model_{model}.png", table_conversion = 'selenium')

df


In [ ]:
interactions = interactions2

m_a = 1
m_dm = 1000

pmag_origin = np.array(interactions[m_a][m_dm]['pmag_origin'])
pmag_cms = np.array(interactions[m_a][m_dm]['pmag_cms'])
nmuons = interactions[m_a][m_dm]['nmuons']

nhits_org = len(pmag_cms)
nhits_cms = len(pmag_cms[pmag_cms>5])

print(f"{nmuons}  {nhits_org}  {nhits_cms}")
print(f"fraction: {nhits_cms/nmuons}")

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.hist(pmag_origin,range=(0,m_dm));

plt.subplot(1,2,2)
plt.hist(pmag_cms[pmag_cms>5],range=(0,m_dm));


plt.figure()
plt.plot(pmag_origin,pmag_cms,'.')
plt.xlim(0,120)
plt.ylim(0,120)